Data Preprocessing
You'll first need to preprocess the CSV data to create the appropriate format for training the DBN. This involves separating the data into features and targets for training and testing.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Load the data
csv_file = 'synthetic_trajectory_data.csv'
data = pd.read_csv(csv_file)

# Extract features and targets
features = ['Timestep', 'Projectile_Position_X', 'Projectile_Position_Y',
            'Projectile_Velocity', 'Projectile_Angle',
            'Interceptor_Position_X', 'Interceptor_Position_Y',
            'Interceptor_Velocity', 'Interceptor_Angle']

# Features for current timestep
X = data[features]

# Add future timestep data to create the target variables
future_features = ['Projectile_Position_X', 'Projectile_Position_Y',
                    'Interceptor_Position_X', 'Interceptor_Position_Y']
y = data[future_features].shift(-1)  # Shift by one timestep to get future positions
y = y[:-1]  # Remove last row with NaN values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Define and Train the DBN
Update the DBN structure to reflect the temporal dependencies and relationships for trajectory prediction.

In [ ]:
from pgmpy.models import DynamicBayesianNetwork as DBN
from pgmpy.estimators import MaximumLikelihoodEstimator

def define_dbn_structure():
    dbn = DBN()
    
    # Temporal edges
    edges = [
        (('Projectile_Position_X', 0), ('Projectile_Position_X', 1)),
        (('Projectile_Position_Y', 0), ('Projectile_Position_Y', 1)),
        (('Interceptor_Position_X', 0), ('Interceptor_Position_X', 1)),
        (('Interceptor_Position_Y', 0), ('Interceptor_Position_Y', 1))
    ]
    dbn.add_edges_from(edges)
    
    # Dependencies between current state variables
    dependencies = [
        (('Projectile_Position_X', 0), ('Interceptor_Position_X', 0)),
        (('Projectile_Position_Y', 0), ('Interceptor_Position_Y', 0)),
        (('Interceptor_Position_X', 0), ('Interceptor_Position_X', 1)),
        (('Interceptor_Position_Y', 0), ('Interceptor_Position_Y', 1))
    ]
    dbn.add_edges_from(dependencies)
    
    return dbn

# Initialize the DBN structure
dbn = define_dbn_structure()

# Fit the DBN using the MaximumLikelihoodEstimator
dbn.fit(X_train, estimator=MaximumLikelihoodEstimator)


Predict and Evaluate
Use the trained DBN to predict the future positions and compare them with the actual future positions.

In [ ]:
from pgmpy.inference import DBNInference

# Initialize the DBN Inference
dbn_infer = DBNInference(dbn)

# Predict and evaluate
predictions = []
for i in range(len(X_test)):
    evidence = X_test.iloc[i].to_dict()
    pred = dbn_infer.forward_inference(variables=[('Projectile_Position_X', 1), ('Projectile_Position_Y', 1),
                                                  ('Interceptor_Position_X', 1), ('Interceptor_Position_Y', 1)],
                                        evidence=evidence)
    predictions.append(pred)

# Convert predictions to DataFrame for comparison
predictions_df = pd.DataFrame(predictions)

# Evaluate the performance (you may need to adapt this based on your exact evaluation criteria)
accuracy = np.mean(np.abs(predictions_df.values - y_test.values) < tolerance)
print(f"Prediction Accuracy: {accuracy:.2f}")
